In [1]:
import pandas as pd
import glob
from afinn import Afinn
from nltk.corpus import stopwords 
from nltk.stem.porter import PorterStemmer
from nltk.tokenize import RegexpTokenizer
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# 현재 디렉터리에 있는 txt 파일을 전부 불러옴! 
pos_review=(glob.glob("../피싱 사례 텍스트 파일/smishing/*.txt"))[0:142]

In [3]:
# Data = pd.read_csv('word_network_csv.csv', engine = "python")

In [4]:
lines_pos=[]
for i in pos_review:
    try:
        f = open(i, 'r')
        temp = f.readlines()[0]
        lines_pos.append(temp)
        f.close()
    except Exception as e:
        continue

In [5]:
len(pos_review)

7

In [6]:
tokenizer = RegexpTokenizer('[\w]+')

In [7]:
stop_words = stopwords.words('english')

In [8]:
count = {} # 동시출현 빈도가 저장될 dict
for line in lines_pos:
    words =  line.lower()
    tokens = tokenizer.tokenize(words)
    stopped_tokens = [i for i in list(set(tokens)) if not i in stop_words+["br"]]
    stopped_tokens2 = [i for i in stopped_tokens if len(i)>1]
    for i, a in enumerate(stopped_tokens2):
        for b in stopped_tokens2[i+1:]:
            if a>b: 
                count[b, a] = count.get((b, a),0) + 1  
            else :
                count[a, b] = count.get((a, b),0) + 1

In [9]:
df=pd.DataFrame.from_dict(count, orient='index')

In [10]:
list1=[]
for i in range(len(df)):
    list1.append([df.index[i][0],df.index[i][1],df[0][i]])

In [11]:
df2=pd.DataFrame(list1, columns=["term1","term2","freq"])

In [12]:
df3=df2.sort_values(by=['freq'],ascending=False)

In [13]:
df3_pos=df3.reset_index(drop=True)

In [14]:
df3_pos.head(40)

,term1,term2,freq
0,phishing,scam,2
1,attacks,phishing,1
2,duped,facebook,1
3,messenger,scam,1
4,facebook,scam,1
5,millions,scam,1
6,facebook,messenger,1
7,messenger,millions,1
8,facebook,millions,1
9,campaign,sms,1


In [15]:
# 결과를 csv 파일로 저장
df = pd.DataFrame.from_dict(df3_pos)
df.to_csv('smishing 연관어 분석.csv')

In [16]:
import networkx as nx
import operator

In [17]:
G_pos=nx.Graph()

In [18]:
for i in range((len(np.where(df3_pos['freq']>10)[0]))):
    G_pos.add_edge(df3_pos['term1'][i], df3_pos['term2'][i], weight=int(df3_pos['freq'][i]))

In [19]:
dgr = nx.degree_centrality(G_pos)
btw = nx.betweenness_centrality(G_pos)
cls = nx.closeness_centrality(G_pos)
egv = nx.eigenvector_centrality(G_pos)

NetworkXPointlessConcept: cannot compute centrality for the null graph

In [ ]:
sorted_dgr = sorted(dgr.items(), key=operator.itemgetter(1), reverse=True)
sorted_btw = sorted(btw.items(), key=operator.itemgetter(1), reverse=True)
sorted_cls = sorted(cls.items(), key=operator.itemgetter(1), reverse=True)
sorted_egv = sorted(egv.items(), key=operator.itemgetter(1), reverse=True)

In [ ]:
G = nx.Graph()

In [ ]:
for i in range(len(sorted_cls)):
    G.add_node(sorted_cls[i][0], nodesize=sorted_dgr[i][1])

In [ ]:
for i in range((len(np.where(df3_pos['freq']>10)[0]))):
    G.add_weighted_edges_from([(df3_pos['term1'][i], df3_pos['term2'][i],int(df3_pos['freq'][i]))])

In [ ]:
sizes = [G.nodes[node]['nodesize']*500 for node in G]

In [ ]:
options = {
    'edge_color': '#FFDEA2',
    'width': 1,
    'with_labels': True,
    'font_weight': 'regular',
}

In [ ]:
nx.draw(G, node_size=sizes, pos=nx.spring_layout(G, k=3.5, iterations=50), **options)
ax = plt.gca()
ax.collections[0].set_edgecolor("#555555") 
plt.show()